In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pulp
from mymodule import *
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df11 = pd.read_excel('../excels/1.xlsx', sheet_name='乡村的现有耕地')
# print(df11)
df12 = pd.read_excel('../excels/1.xlsx', sheet_name='乡村种植的农作物')
df12.fillna(method='ffill', inplace=True)
# print(df12)
df21 = pd.read_excel('../excels/2.xlsx', sheet_name='2023年的农作物种植情况')
df21.fillna(method='ffill', inplace=True)
# print(df21)
df22 = pd.read_excel('../excels/2.xlsx', sheet_name='2023年统计的相关数据')
# 计算销售单价平均值
def calculate_average_price(price_str):
    prices = list(map(float, price_str.split('-')))
    return sum(prices) / len(prices)
df22['销售单价平均值/(元/斤)'] = df22['销售单价/(元/斤)'].apply(calculate_average_price)
# print(df22)
df2023_result = pd.read_excel('../excels/附件3/2023_result.xlsx')
# 去除列名末尾的空格。很奇怪
df2023_result.columns = [column.rstrip(' ') for column in df2023_result.columns]
# print(df2023_result)

In [12]:
result1 = df21.groupby(['作物编号', '地块类型', '种植季次'])['种植面积/亩'].sum().reset_index()
result1.columns = ['作物编号', '地块类型', '种植季次', '种植面积/亩']
merged_df = pd.merge(result1, df22[['作物编号', '作物名称','作物类型' , '地块类型', '种植季次', '亩产量/斤', '种植成本/(元/亩)', '销售单价/(元/斤)']], on=['作物编号', '种植季次', '地块类型'], how='left')
merged_df['单地块类型总产量/斤'] = merged_df['种植面积/亩'] * merged_df['亩产量/斤']
merged_df['销售单价平均值'] = merged_df['销售单价/(元/斤)'].apply(calculate_average_price)
merged_df['利润'] = (merged_df['销售单价平均值'] * merged_df['单地块类型总产量/斤']) - (merged_df['种植成本/(元/亩)'] * merged_df['种植面积/亩'])
merged_df['每亩利润'] = merged_df['利润'] / merged_df['种植面积/亩']
# print(merged_df)
total_profit_2023 = merged_df.groupby(['作物编号', '作物名称'])[['利润', '每亩利润']].sum().reset_index()
total_profit_2023.columns = ['作物编号', '作物名称', '总利润/元', '每亩总利润/元']
# print(total_profit)
total_yield_2023 = merged_df.groupby(['作物编号', '作物名称'])['单地块类型总产量/斤'].sum().reset_index()
total_yield_2023.columns = ['作物编号', '作物名称', '总产量/斤']
# print(total_yield_2023)

In [13]:
fields = create_fields(df11)
# for field in fields:
#     print(field)
new_fields = create_new_fields(fields)
# for field in new_fields:
#     print(field)
crops = create_crops(df12, df22)
# for crop in crops:
#     print(crop)    

In [ ]:
# 示例调用

fieldtest = new_fields[-0]
croptest = crops[0]
print(fieldtest.field_name, fieldtest.season, croptest.crop_name)
yield_value = get_yield(fieldtest, croptest)
cost_value = get_cost(fieldtest, croptest)
print(f"Yield for {croptest.crop_name} in {fieldtest.field_name}: {yield_value}")
print(f"Cost for {croptest.crop_name} in {fieldtest.field_name}: {cost_value}")
total_yield = get_expected_sales(croptest, total_yield_2023)
print(f"Total yield for {croptest.crop_name} in 2023: {total_yield}")
print(f"{fieldtest.field_name} {croptest.crop_name} {check_crop_constraints(fieldtest, croptest)}")
print(check_rotation_constraints(new_fields[0], crops[0], df2023_result)) # 黄豆2023年第一季在A1地块未种植
print(check_rotation_constraints(new_fields[0], crops[5], df2023_result)) # 小麦2023年第一季在A1地块种植

In [16]:
# 创建线性规划问题
prob = pulp.LpProblem("Crop_Planting_Optimization", pulp.LpMaximize)

# 创建决策变量
variables = {}
binary_variables = {}
for field in new_fields:
    for crop in crops:
        var_name = f"{field.field_name}_{crop.crop_name}_{field.season}"
        # print(var_name)
        variables[(field.field_name, crop.crop_name, field.season)] = pulp.LpVariable(var_name, lowBound=0)
        binary_var_name = f"binary_{var_name}"
        binary_variables[(field.field_name, crop.crop_name, field.season)] = pulp.LpVariable(binary_var_name, cat='Binary')
   
# 创建实际销售量变量
actual_sales = {}
for crop in crops:
    actual_sales[crop.crop_name] = pulp.LpVariable(f"actual_sales_{crop.crop_name}", lowBound=0)

# 创建超额部分变量
excess_yield = {}
for crop in crops:
    excess_yield[crop.crop_name] = pulp.LpVariable(f"excess_yield_{crop.crop_name}", lowBound=0)

In [17]:
# 定义目标函数的各个部分

# 收入部分：实际销售量 * 作物价格
revenue = pulp.lpSum([
    actual_sales[crop.crop_name] * crop.crop_price
    for crop in crops
])

# 成本部分：种植成本
cost = pulp.lpSum([
    variables[(field.field_name, crop.crop_name, field.season)] * get_cost(field, crop)
    for field in new_fields for crop in crops
])

# 超额部分处理：超额部分 * 作物价格 * 系数
k = 0  # 系数
excess_handling = pulp.lpSum([
    excess_yield[crop.crop_name] * crop.crop_price * k
    for crop in crops
])

# 总利润 = 收入 - 成本 + 超额部分处理
profit = revenue - cost + excess_handling

# 将目标函数添加到问题中
prob += profit

# print(profit)

In [18]:
# 添加约束条件
# 地块面积限制
for field in new_fields:
    prob += pulp.lpSum([variables[(field.field_name, crop.crop_name, field.season)] for crop in crops]) <= field.field_area

# 作物种植限制
for field in new_fields:
    for crop in crops:
        if not check_crop_constraints(field, crop):
            prob += variables[(field.field_name, crop.crop_name, field.season)] == 0

# 重茬限制
for field in new_fields:
    for crop in crops:
        if not check_rotation_constraints(field, crop, df2023_result):
            prob += variables[(field.field_name, crop.crop_name, field.season)] == 0

# 豆类作物种植要求
# for field in new_fields:
#     prob += pulp.lpSum([variables[(field.field_name, crop.crop_name, field.season)] 
#                         for crop in crops if crop.crop_type in ['粮食（豆类）', '蔬菜（豆类）']]) >= 1

# 实际销售量约束
for crop in crops:
    actual_yield = pulp.lpSum([
        variables[(field.field_name, crop.crop_name, field.season)] * get_yield(field, crop)
        for field in new_fields
    ])
    expected_sales = get_expected_sales(crop, total_yield_2023)
    prob += actual_sales[crop.crop_name] <= actual_yield
    prob += actual_sales[crop.crop_name] <= expected_sales

    # 超额部分约束
    prob += excess_yield[crop.crop_name] == actual_yield - expected_sales # 这里不是>=，应该是==？！
    prob += excess_yield[crop.crop_name] >= 0

# # 作物在单个地块某一季节种植面积不宜太小
# min_area_percent = 0.2
# for field in new_fields:
#     for crop in crops:
#         var = variables[(field.field_name, crop.crop_name, field.season)]
#         binary_var = binary_variables[(field.field_name, crop.crop_name, field.season)]
#         min_area = min_area_percent * field.field_area
#         prob += var >= min_area * binary_var
#         prob += var <= field.field_area * binary_var

# 作物种植不宜太分散
max_plots = 4
for crop in crops:
    for field_type in set(field.field_type for field in new_fields):
        if field_type not in ['普通大棚', '智慧大棚']:
            prob += pulp.lpSum([
                binary_variables[(field.field_name, crop.crop_name, field.season)]
                for field in new_fields if field.field_type == field_type
            ]) <= max_plots

In [19]:
# 求解问题
prob.solve()
print("Status:", pulp.LpStatus[prob.status])
# print("Objective:", pulp.value(prob.objective))

# 打印所有变量及其值
# for v in prob.variables():
#     print(v.name, "=", v.varValue)

# for v in prob.variables():
#     if v.varValue > 0:
#         print(v.name, "=", v.varValue)

# 打印所有约束条件
# for name, constraint in prob.constraints.items():
#     print(f"{name}: {constraint}")

# 打印模型的详细信息
# print(prob)

Status: Optimal


In [20]:
# 更新 new_fields 列表中的种植情况
for v in variables.values():
    # 解析 v.name 得到 field_name, crop_name 和 season
    field_name, crop_name, season = v.name.split('_')
    # 找到对应的 Field 对象
    for field in new_fields:
        if field.field_name == field_name and field.season == season:
            # 向 planted_crop 元素追加一个二元列表
            field.planted_crop.append([crop_name, v.varValue])
            break
# 打印结果
# for field in new_fields:
#     print(field.field_name, field.season, field.planted_crop)

df2024 = pd.read_excel('../excels/附件3/2024.xlsx')
# 将读取到的表格中的NaN取值为0
df2024.fillna(0, inplace=True)
# print(df2024)
df2024.columns = [column.rstrip(' ') for column in df2024.columns]
# print(df2024.columns)
# 遍历 new_fields，更新 df2024
for field in new_fields:
    season = "第一季" if field.season == "单季" else field.season
    field_name = field.field_name

    # 找到对应的行
    row_index = df2024[(df2024.iloc[:, 0] == season) & (df2024.iloc[:, 1] == field_name)].index

    if not row_index.empty:
        row_index = row_index[0]
        # 遍历 planted_crop，更新 df2024
        for crop in field.planted_crop:
            crop_name = crop[0]
            crop_area = crop[1]

            # 找到对应的列
            col_index = df2024.columns.get_loc(crop_name)

            # 更新单元格
            df2024.iloc[row_index, col_index] = crop_area
# 在最后一列添加每一行所有种植面积的求和
df2024['总种植面积'] = df2024.iloc[:, 2:].sum(axis=1)

# 打印更新后的 df2024
# print(df2024)

# 保存更新后的 df2024 到 Excel 文件
df2024.to_excel('../excels/附件3/2024_result.xlsx', index=False)

e:\Users\Timothy\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
